In [1]:
import numpy as np
import pandas as pd
import json
from datetime import datetime

# Leemos dataset
# Lee el archivo JSON
with open("./data2/df_bookings_9.json", encoding="'ISO-8859-1'") as f:
    data = json.load(f)

# Normaliza los datos JSON en un DataFrame (para leer la fecha correctamente)
df_bookings = pd.json_normalize(data)

# Leemos tabla events
df_events = pd.read_csv("./data2/DT2023.events_2.csv")
df_events.set_index("_id", inplace=True)

# Preparamos dataset con los campos necesarios
df=df_bookings.loc[:,["userId","eventId", "createdAt.$date"]]
df.rename(columns={"createdAt.$date":"createdAt"}, inplace=True)

startDate=[]
capacity=[]
for id in df["eventId"]:
    startDate.append(df_events.loc[id, "startDate"])
# for i in df.index:
#     startDate.append(df_events.loc[df.loc[i, "eventId"], "startDate"])

df["startDate"]=startDate

################################################## FORMULA ####################################################
# def predictor_asistencia(df):

# Convertimos campo fecha a asistencias por dia
df["createdAt"]=pd.to_datetime(df["createdAt"])
df["startDate"]=pd.to_datetime(df["startDate"])
df["dias"]=(df["startDate"]-df["createdAt"]).dt.days

# Creamos una columna por dia y incluimos el count
df=df.pivot_table(index="eventId", values="userId", columns=["dias"], aggfunc="count", fill_value=0)

# Conseguimos el acumulado
for i in range(2, len(df.columns)+1):
    df[i] = df[i-1] + df[i]

# Eliminamos valores anómalos usando el criterio de box plot y rangos intercuartílicos
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filtra el DataFrame para eliminar filas con valores fuera de los límites
filtered_df = df[(df >= lower_bound) & (df <= upper_bound)].dropna()

# Normalizamos
from sklearn.preprocessing import MinMaxScaler
max_norm= max(filtered_df)  # Guardamos el máximo para luego desnormalizar
scaler=MinMaxScaler()
scaler.fit(filtered_df.T)
df_norm = pd.DataFrame(scaler.fit_transform(df.T).T, columns=df.columns)

# Calculamos la media de cada columna
column_means = filtered_df.mean()
X = column_means.to_numpy()

# Aplicamos Regresión Polinómica a los valores de ese array
grado_polinomio = 4
coefficients = np.polyfit(np.arange(len(X)), X, grado_polinomio)
polynomial_function = np.poly1d(coefficients)

# La función polinómica ahora está definida y puede ser evaluada en cualquier punto
print(polynomial_function)

c:\Users\sergi\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


          4           3          2
0.000232 x - 0.01895 x + 0.4273 x + 1.313 x + 1.367
